# Generate Spatially Varying Magnitude Errors According to Observing Conditions

last run successfully: Feb 9, 2026

The ObsCondition degrader can be used to generate spatially-varying photometric errors using input survey condition maps in `healpix` format, such as survey coadd depth, airmass, sky brightness etc. The photometric error is computed by `photerr.LsstErrorModel`, based on the LSST Overview Paper:
https://arxiv.org/abs/0805.2366. 

The degrader assigns each object in the input catalogue with a pixel within the survey footprint and computes the magnitude error (SNR) on each pixel.
The degrader takes the following arguments:

- `nside`: nside used for the HEALPIX maps.
- `mask`: Path to the mask covering the survey footprint in HEALPIX format. Notice that all negative values will be set to zero.
- `weight`: Path to the weights HEALPIX format, used to assign sample galaxies in pixels. Default is weight="", which uniform weighting.
- `tot_nVis_flag`: If `nVisYr` is provided in `map_dict` (see below), this flag indicates whether the map shows the total number of visits in nYrObs (`tot_nVis_flag=True`), or the average number of visits per year (`tot_nVis_flag=False`). The default is set to `True`.   
- `random_seed`: A random seed for reproducibility.
- `map_dict`: A dictionary that contains the paths to the survey condition maps in HEALPIX format. This dictionary uses the same arguments as LSSTErrorModel. The following arguements, if supplied, may contain either a single number (as in the case of LSSTErrorModel), or a path to the corresponding survey condition map in `healpix` format:`m5`, `nVisYr`, `airmass`, `gamma`, `msky`, `theta`, `km`, and `tvis`. Notice that *except* `airmass` and `tvis`, for all other arguements, numbers/paths for *specific bands* should be passed. Other `LsstErrorModel` parameters can also be passed in this dictionary (e.g. a necessary one may be `nYrObs` for the survey condition maps; the default value is 10 years, although most  may be interested in early data releases). If any arguement is not passed, the default value in https://arxiv.org/abs/0805.2366 is adopted. Example:
```json
{
   "m5": {"u": "path", ...}, 
   "theta": {"u": "path", ...},
}
```

Argument defaults are determined by the defaults of the `LsstErrorModel` in [PhotErr](https://github.com/jfcrenshaw/photerr).

In this quick notebook we'll generate the photometric error based on the DC2 Y5 LSST median $5\sigma$ depth in $i$-band generated by OpSim `minion_1016` database using the Rubin Observatory Metrics Analysis Framework (MAF).


In [ ]:
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rail.interactive as ri
from rail.utils.path_utils import find_rail_file

Let's generate some fake data.

In [ ]:
# Fake data with same magnitude in each band
i = np.ones(50_000) * 23.0
u = np.full_like(i, 23.0, dtype=np.double)
g = np.full_like(i, 23.0, dtype=np.double)
r = np.full_like(i, 23.0, dtype=np.double)
y = np.full_like(i, 23.0, dtype=np.double)
z = np.full_like(i, 23.0, dtype=np.double)
redshift = np.random.uniform(size=len(i)) * 2

In [ ]:
mockdict = {}
for label, item in zip(
    ["redshift", "u", "g", "r", "i", "z", "y"], [redshift, u, g, r, i, z, y]
):
    mockdict[f"{label}"] = item

In [ ]:
data = pd.DataFrame(mockdict)
data.head()

In [ ]:
data_degraded = ri.creation.degraders.observing_condition_degrader.obs_condition(
    sample=data,
    map_dict={
        "m5": {
            "i": find_rail_file(
                "examples_data/creation_data/data/survey_conditions/minion_1016_dc2_Median_fiveSigmaDepth_i_and_nightlt1825_HEAL.fits"
            ),
        },
        "nYrObs": 5.0,
    },
)

In [ ]:
data_degraded["output"].head()

We see that extra columns containing the magnitude errors: `u_err`, `g_err`... have been added to the catalogue. Notice that since we have only provided the limiting magnitude for $i$-band, the errors in all other bands except $i$ are computed using the default parameters in `LsstErrorModel` (see: https://github.com/jfcrenshaw/photerr/blob/main/photerr/lsst.py). 

The last column shows the pixel of the survey condition map that is assigned to each object. 

We can check if the spatial dependence has been implemented by looking at the SNR at different area of the sky, and compare that with the $i$-band depth:

In [ ]:
mask = hp.read_map(
    find_rail_file(
        "examples_data/creation_data/data/survey_conditions/DC2-mask-neg-nside-128.fits"
    )
)
weight = hp.read_map(
    find_rail_file(
        "examples_data/creation_data/data/survey_conditions/DC2-dr6-galcounts-i20-i25.3-nside-128.fits"
    )
)
Med_5sd_i = hp.read_map(
    find_rail_file(
        "examples_data/creation_data/data/survey_conditions/minion_1016_dc2_Median_fiveSigmaDepth_i_and_nightlt1825_HEAL.fits"
    )
)

In [ ]:
# Set negative values in mask to zero
mask[mask < 0] = 0

In [ ]:
# Compute the average SNR in each pixel
avg_SNR_i = np.zeros(len(mask))
for pix, pix_cat in (data_degraded["output"]).groupby("pixel"):
    avg_SNR_i[pix] = np.mean((pix_cat["i"] / pix_cat["i_err"]).to_numpy())

In [ ]:
# View the healpix map

fig, axarr = plt.subplots(1, 3, figsize=[12, 6])

plt.sca(axarr[0])
hp.gnomview(
    weight * mask / sum(weight),
    rot=(62, -36.5, 0),
    xsize=100,
    ysize=100,
    reso=16,
    title="weight",
    hold=True,
)
plt.sca(axarr[1])
hp.gnomview(
    Med_5sd_i * mask,
    rot=(62, -36.5, 0),
    xsize=100,
    ysize=100,
    reso=16,
    title="5sigmadepth i",
    hold=True,
)
plt.sca(axarr[2])
hp.gnomview(
    avg_SNR_i,
    rot=(62, -36.5, 0),
    xsize=100,
    ysize=100,
    reso=16,
    title="avg SNR i",
    min=1400,
    max=1750,
    hold=True,
)

Now if we want to change any of the default settings, we can supply them in `ObsCondition.make_stage()`. In this example, instead of supplying the median $5\sigma$ depth in $i$-band, we supply the median airmass in $i$-band. In this case, the $i$-band limiting magnitude `m5` will be computed explicitly (notice that if `m5` is also supplied, then it will overwrite the explicitly computed `m5`).

In [ ]:
data_degraded_airmass = ri.creation.degraders.observing_condition_degrader.obs_condition(
    sample=data,
    map_dict={
        "airmass": find_rail_file(
            "examples_data/creation_data/data/survey_conditions/minion_1016_dc2_Median_airmass_i_and_nightlt1825_HEAL.fits"
        ),
        "m5": {
            "i": find_rail_file(
                "examples_data/creation_data/data/survey_conditions/minion_1016_dc2_Median_fiveSigmaDepth_i_and_nightlt1825_HEAL.fits"
            ),
        },
        "nYrObs": 5.0,
    },
)

In [ ]:
data_degraded_airmass["output"].head()

Again, we can examine whether the spatial dependence is indeed applied. Here, `LsstErrorModel` does not have band-dependent airmass, so it affects all bands. The default airmass is $X=1.2$, but the input median airmass is more optimistic, thus reducing the magnitude errors.

In [ ]:
Med_airmass_i = hp.read_map(
    find_rail_file(
        "examples_data/creation_data/data/survey_conditions/minion_1016_dc2_Median_airmass_i_and_nightlt1825_HEAL.fits"
    )
)

Compute the average SNR in each pixel for i and r bands:

In [ ]:
avg_SNR_i_airmass = np.zeros(len(mask))
avg_SNR_r_airmass = np.zeros(len(mask))
for pix, pix_cat in (data_degraded_airmass["output"]).groupby("pixel"):
    avg_SNR_i_airmass[pix] = np.mean((pix_cat["i"] / pix_cat["i_err"]).to_numpy())
    avg_SNR_r_airmass[pix] = np.mean((pix_cat["r"] / pix_cat["r_err"]).to_numpy())

View the healpix map:

In [ ]:
fig, axarr = plt.subplots(1, 3, figsize=[12, 6])

plt.sca(axarr[0])
hp.gnomview(
    Med_airmass_i * mask,
    rot=(62, -36.5, 0),
    xsize=100,
    ysize=100,
    reso=16,
    title="airmass i",
    hold=True,
)
plt.sca(axarr[1])
hp.gnomview(
    avg_SNR_i_airmass,
    rot=(62, -36.5, 0),
    xsize=100,
    ysize=100,
    reso=16,
    title="avg SNR i",
    min=2240,
    max=2280,
    hold=True,
)

plt.sca(axarr[2])
hp.gnomview(
    avg_SNR_r_airmass,
    rot=(62, -36.5, 0),
    xsize=100,
    ysize=100,
    reso=16,
    title="avg SNR r",
    min=2930,
    max=2970,
    hold=True,
)

In both cases, we see a negative correlation between the airmass and the SNR in $i$ and $r$ bands, as expected.